In [15]:
from collections import namedtuple
from pathlib import Path

import numpy as np

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR / "2022-18.txt"

with INPUTFILE.open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

test = [
    "2,2,2",
    "1,2,2",
    "3,2,2",
    "2,1,2",
    "2,3,2",
    "2,2,1",
    "2,2,3",
    "2,2,4",
    "2,2,6",
    "1,2,5",
    "3,2,5",
    "2,1,5",
    "2,3,5",
]

# inputs = test

inputs[0:8]


['15,15,12',
 '11,3,10',
 '5,2,11',
 '17,12,13',
 '6,4,15',
 '16,13,5',
 '9,16,13',
 '6,5,4']

In [16]:
Location = namedtuple("Location", ["x", "y", "z"])

locations = []
for line in inputs:
    x, y, z = list(map(int, line.split(",")))
    location = Location(x, y, z)
    locations.append(location)

print(locations[0:8])


[Location(x=15, y=15, z=12), Location(x=11, y=3, z=10), Location(x=5, y=2, z=11), Location(x=17, y=12, z=13), Location(x=6, y=4, z=15), Location(x=16, y=13, z=5), Location(x=9, y=16, z=13), Location(x=6, y=5, z=4)]


In [17]:
for fn in (min, max):
    print(f"{fn(map(lambda l:l.x, locations))}")
    print(f"{fn(map(lambda l:l.y, locations))}")
    print(f"{fn(map(lambda l:l.z, locations))}")

DIMENSION = max(map(max, locations)) + 1 + 2
DIMENSION


0
0
0
19
19
19


22

In [18]:
model = np.zeros((DIMENSION, DIMENSION, DIMENSION), dtype=np.int8)

AIR = 0
ROCK = 1

for location in locations:
    model[location.z+1, location.y+1, location.x+1] = ROCK


In [19]:
# print(model[10, 10, :])
# print(np.diff(model[10, 10, :]))
# print(np.abs(np.diff(model[10, 10, :])))
# print(np.sum(np.abs(np.diff(model[10, 10, :]))))

def exposed_faces(data: np.ndarray) -> int:

    faces = 0
    for i in range(0, DIMENSION):
        for j in range(0, DIMENSION):

            count = np.sum(np.abs(np.diff(data[i,j,:])))
            if count %2 == 1:
                print(f"[{i-1},{j-1},:] - {data[i,j,:]}")
            faces += count

            count = np.sum(np.abs(np.diff(data[i,:,j])))
            if count %2 == 1:
                print(f"[{i-1},:,{j-1}] - {data[i,:,j]}")
            faces += count

            count = np.sum(np.abs(np.diff(data[:,i,j])))
            if count %2 == 1:
                print(f"[:,{i-1},{j-1}] - {data[:,i,j]}")
            faces += count

    return faces

print(f"Part 1: {exposed_faces(model)} faces are exposed")


Part 1: 3496 faces are exposed


In [20]:
volume = np.array(model)

EXP = 8
# mark all margins as exposed
volume[0,:,:] = EXP
volume[:,0,:] = EXP
volume[:,:,0] = EXP
volume[-1,:,:] = EXP
volume[:,-1,:] = EXP
volume[:,:,-1] = EXP

# print("\n***** Volume with margins marked exterior: *****\n")
# for z in range(DIMENSION):
#     print(f"{z}: {volume[z,:,:]}")

seeds = set()
for i in range(1,DIMENSION-1):
    for j in range(1, DIMENSION-1):
        if volume[1,i,j] != ROCK:
            volume[1,i,j] = EXP
            seeds.add(Location(j,i,1))
        if volume[-2,i,j] != ROCK:
            volume[-2,i,j] = EXP
            seeds.add(Location(j,i,-2))
        if volume[i,1,j] != ROCK:
            volume[i,1,j] = EXP
            seeds.add(Location(j,1,i))
        if volume[i,-2,j] != ROCK:
            volume[i,-2,j] = EXP
            seeds.add(Location(j,-2,i))
        if volume[i,j,1] != ROCK:
            volume[i,j,1] = EXP
            seeds.add(Location(1,j,i))
        if volume[i,j,-2] != ROCK:
            volume[i,j,-2] = EXP
            seeds.add(Location(-2,j,i))

# print("\n***** Volume with seeds identified: *****\n")
# for z in range(DIMENSION):
#     print(f"{z}: {volume[z,:,:]}")

while len(seeds) > 0:
    test = seeds.pop()
    if volume[test.z-1,test.y,test.x] == AIR:
        volume[test.z-1,test.y,test.x] = EXP
        seed = Location(test.x, test.y, test.z-1)
        seeds.add(seed)
    if volume[test.z+1,test.y,test.x] == AIR:
        volume[test.z+1,test.y,test.x] = EXP
        seed = Location(test.x, test.y, test.z+1)
        seeds.add(seed)
    if volume[test.z,test.y-1,test.x] == AIR:
        volume[test.z,test.y-1,test.x] = EXP
        seed = Location(test.x,test.y-1,test.z)
        seeds.add(seed)
    if volume[test.z,test.y+1,test.x] == AIR:
        volume[test.z,test.y+1,test.x] = EXP
        seed = Location(test.x,test.y+1,test.z)
        seeds.add(seed)
    if volume[test.z,test.y,test.x-1] == AIR:
        volume[test.z,test.y,test.x-1] = EXP
        seed = Location(test.x-1,test.y,test.z)
        seeds.add(seed)
    if volume[test.z,test.y,test.x+1] == AIR:
        volume[test.z,test.y,test.x+1] = EXP
        seed = Location(test.x+1,test.y,test.z)
        seeds.add(seed)

# volume[target.z, target.y, target.x] = 42
exterior = volume == EXP
exterior = exterior.astype(dtype=np.int8)

print(f"Part 2: {exposed_faces(exterior)} faces are externally exposed")

# print("\n***** Model: *****\n")
# for z in range(DIMENSION):
#     print(f"{z}: {model[z,:,:]}")

# print("\n***** Volume: *****\n")
# for z in range(DIMENSION):
#     print(f"{z}: {volume[z,:,:]}")

# print("\n***** Exterior: *****\n")
# for z in range(DIMENSION):
#     print(f"{z}: {exterior[z,:,:]}")


Part 2: 2064 faces are externally exposed
